## Descargar datos de JVP   

In [18]:
import MySQLdb
from datetime import datetime, timedelta
import pandas as pd

db = MySQLdb.connect(host="192.168.0.12",    # tu host, usualmente localhost
                     user="operario",         # tu usuario
                     passwd="",  # tu password
                     db="dbp8100")        # el nombre de la base de datos

# Debes crear un objeto Cursor. Te permitirá
# ejecutar todos los queries que necesitas
cur = db.cursor()


In [20]:
fecha_actual = datetime.now()
fecha_actual = fecha_actual.strftime("%Y-%m-%d")

In [9]:
fecha_ayer = datetime.now()
fecha_ayer = fecha_ayer.strftime('%Y-%m-%d')

In [25]:
cur.execute("""
    SELECT 
        productox.Codigo AS Codigo, 
        productox.Nombre AS Producto, 
        SUM(dcaptura.Valor) AS Dosificado 
    FROM 
        dbp8100.dcaptura AS dcaptura
    JOIN 
        dbp8100.tareaseje AS tareaseje ON dcaptura.IDT = tareaseje.NroID
    JOIN 
        dbp8100.productox AS productox ON productox.NroID = dcaptura.IDP 
    WHERE 
        tareaseje.Fecha >= %s AND 
        tareaseje.Fecha <= %s 
    GROUP BY 
        productox.Nombre
""", (fecha_ayer, fecha_actual))

13

In [26]:
# Obtener los resultados
resultados = cur.fetchall()
print(resultados)

(('B-07', 'AFRECHO ARROZ', 8711.2), ('C-19', 'BENTONITA', 1301.0), ('C-12', 'CARBONATO DE CALCIO', 862.2), ('A-06', 'CASCARA ARROZ', 1892.0), ('A-04', 'EXP SOJA', 5327.0), ('A-02', 'MAIZ', 18541.0), ('A-01', 'PELLET AFRECHILLO DE TRIGO', 28557.0), ('A-03', 'PELLET DE GIRASOL', 1733.0), ('C-11', 'SAL', 1117.4), ('D-13', 'SUPLEFEED23', 114.2), ('C-19', 'tK DESARROLLO 2,5%', 245.9), ('A-05', 'TRIGO (para balanceado)', 3757.0), ('C-10', 'UREA', 333.8))


In [27]:

# Obtener los nombres de las columnas
columnas = [desc[0] for desc in cur.description]

# Crear un DataFrame de pandas
df = pd.DataFrame(resultados, columns=columnas)

# Cerrar el cursor y la conexión
cur.close()
db.close()

# Mostrar el DataFrame
print(df)

   Codigo                    Producto  Dosificado
0    B-07               AFRECHO ARROZ      8711.2
1    C-19                   BENTONITA      1301.0
2    C-12         CARBONATO DE CALCIO       862.2
3    A-06               CASCARA ARROZ      1892.0
4    A-04                    EXP SOJA      5327.0
5    A-02                        MAIZ     18541.0
6    A-01  PELLET AFRECHILLO DE TRIGO     28557.0
7    A-03           PELLET DE GIRASOL      1733.0
8    C-11                         SAL      1117.4
9    D-13                 SUPLEFEED23       114.2
10   C-19          tK DESARROLLO 2,5%       245.9
11   A-05     TRIGO (para balanceado)      3757.0
12   C-10                        UREA       333.8


## Cargar códigos

In [13]:
df_cod = pd.read_excel('codigos.xlsx')
df_cod.head(10)

,Producto,Cod
0,ACEITE DE SOJA,NaN
1,ACID-PACK,NaN
2,ACTIGEN,NaN
3,ACTIVATE (METIONINA),NaN
4,ADITIVO BIOTOX,NaN
5,AFRECHILLO DE TRIGO,AFTR0
6,AFRECHO ARROZ,AA0
7,ALFAZOLE,NaN
8,ALL-MIC,NaN
9,ALLMIC,NaN


In [14]:
df_result = pd.merge(df, df_cod, on='Producto')
df_result.head(10)

,Codigo,Producto,Dosificado,Cod
0,B-07,AFRECHO ARROZ,8711.2,AA0
1,C-19,BENTONITA,1301.0,NaN
2,C-12,CARBONATO DE CALCIO,862.2,NaN
3,A-06,CASCARA ARROZ,1892.0,CA0
4,A-04,EXP SOJA,5327.0,NaN
5,A-02,MAIZ,18541.0,19
6,A-01,PELLET AFRECHILLO DE TRIGO,28557.0,PET0
7,A-03,PELLET DE GIRASOL,1733.0,NaN
8,C-11,SAL,1117.4,NaN
9,D-13,SUPLEFEED23,114.2,NaN


In [15]:
df_result.dropna(inplace=True)
df_result.drop(['Codigo','Producto'], axis=1, inplace=True)
df_result = df_result[['Cod','Dosificado']]
df_result.head(10)

,Cod,Dosificado
0,AA0,8711.2
3,CA0,1892.0
5,19,18541.0
6,PET0,28557.0


In [16]:
import os

# Especificamos la ruta de la carpeta que deseamos crear
ruta_carpeta = 'ReporteDiario'

# Verificamos si la carpeta ya existe
if not os.path.exists(ruta_carpeta):
    # Creamos la carpeta si no existe
    os.mkdir(ruta_carpeta)
    print(f"La carpeta '{ruta_carpeta}' ha sido creada con éxito.")
else:
    print(f"La carpeta '{ruta_carpeta}' ya existe.")

La carpeta 'ReporteDiario' ya existe.


In [17]:
# Obtenemos la fecha actual
fecha_actual = datetime.now().strftime("%Y-%m-%d")

# Exportamos el DataFrame a un archivo de Excel
df_result.to_excel(f"ReporteDiario/{fecha_actual}.xlsx", index=False)